In [ ]:
import numpy as np
import pygame
import sys
from boardparams import *

def create_board():
    #Creates and populates a 2-D array filled with zero of size ROW_COUNT x COLUMN_COUNT
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))
    return board

def drop_piece(board, row, selection, piece):
    #Drops a piece in the selected location (column)
    board[row][selection] = piece
    return board

def is_column_empty(board, selection):
    #Checks if top row is filled. If it is, then this function will not return True and will prompt user later in the 
    #code to pick a different column that is empty
    if board[ROW_COUNT-1][selection] == 0:
        return True
    
def get_next_open_row(board, selection):
    #Implements "gravity". For any selected column, this will start counting from the bottom and return the lowest row 
    #that is empty (i.e. == 0). This empty row can then be filled by either player
    for row in range(ROW_COUNT):
        if board[row][selection] == 0:
            return row
        
def win_condition(board, piece):
    for c in range(COLUMN_COUNT-4):
        for r in range(ROW_COUNT-4):
            #Horizontal win
            if board[r][c] == piece:
                if board[r][c+1] == piece:
                    if board[r][c+2] == piece:
                        if board[r][c+3] == piece:
                            return True
            #Vertical win
                elif board[r+1][c] == piece:
                    if board[r+2][c] == piece:
                        if board[r+3][c] == piece:
                            return True
                            
            #Positively sloping diagonal win
                elif board[r+1][c+1] == piece:
                    if board[r+2][c+2] == piece:
                        if board[r+3][c+3] == piece:
                            return True
                        
            #Negatively sloping diagonal win           
    for c in range(COLUMN_COUNT-3,COLUMN_COUNT):
        for r in range(ROW_COUNT-3):    
            if board[r][c] == piece:
                if board[r+1][c-1] == piece:
                    if board[r+2][c-2] == piece:
                        if board[r+3][c-3] == piece:
                            return True
                        
def draw_board(board):
    #This flip is necessary because numpy arrays read from top to bottom, while connect four logic requires us to count
    #from bottom to the top. This flip maintains the correct orientation for displaying the game.
    board = np.flip(board,0)
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            #Draws a blue rectangle that's one row shorter than the entire screen size
            pygame.draw.rect(screen, BLUE, pygame.Rect(c*SQUARE_SIZE, (r+1)*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
            if board[r][c] == 0:
                #Fills a BLACK circle at each row and each column of the size specified by CIRCLE_RADIUS
                pygame.draw.circle(screen, BLACK, (c*SQUARE_SIZE+OFFSET, (r+1)*SQUARE_SIZE+OFFSET), CIRCLE_RADIUS)
            elif board[r][c] == 1:
                #Fills a RED circle for player 1 in whichever spot player 1 drops a piece or where board[r][c] == 1
                pygame.draw.circle(screen, RED, (c*SQUARE_SIZE+OFFSET, (r+1)*SQUARE_SIZE+OFFSET), CIRCLE_RADIUS)
                #Fills a YELLOW circle for player 1 in whichever spot player 1 drops a piece or where board[r][c] == 1
            elif board[r][c] == 2:
                pygame.draw.circle(screen, YELLOW, (c*SQUARE_SIZE+OFFSET, (r+1)*SQUARE_SIZE+OFFSET), CIRCLE_RADIUS)
    #updates display every time something happens
    pygame.display.update()   

In [ ]:
#Initialize the game
game_over = False
board = create_board()
turn = 0
pygame.init()
screen = pygame.display.set_mode(SIZE)
myfont = pygame.font.SysFont("Tahoma", 40)

#Keep playing until game doesn't end in win or draw
while not game_over: 
    draw_board(board)
    for event in pygame.event.get():
        #Allows pygame to exit gracefully
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
            break
        #Detects mouse clicks and assigns mouse click location to a particular column by dividing the click location
        #by square size which returns an integer between 0 and 6
        elif event.type == pygame.MOUSEBUTTONDOWN:
            pos_x = int(event.pos[0] / SQUARE_SIZE)
            selection = pos_x
            
            turn +=1
            #Checks if number of turns is greater than board dimensions in which case match has ended in a draw
            if turn > COLUMN_COUNT*ROW_COUNT:  
                draw_board(board)
                #Puts up a message that game is drawn
                label = myfont.render(f"Game Over! Match Drawn!", 1, WHITE)
                screen.blit(label, (0, 0))
                pygame.display.update()
                #Allows game to exit gracefully
                pygame.time.delay(5000)
                pygame.quit()
                sys.exit()
                break
            else:
                #Odd turns belong to player 1
                if turn % 2 == 1:
                    piece = 1
                #Even turns belong to player 2
                else:
                    piece = 2
                print(np.flip(board,0))
            #This pauses the game at a point when a player tries to drop a piece in a column that is full 
            #and will only allow the game to proceed when a new column is picked
            while not is_column_empty(board, selection):
                #Puts up a message prompting player to pick a different column
                label = myfont.render(f"Player {piece} pick a different column", 1, WHITE)
                screen.blit(label, (0, 0))
                pygame.display.update()
                pygame.time.delay(100)
                #Overwrites the message with a black rectangle which resets the display to its previous state
                #Without this statement new text will keep appearing on top of old text which looks ugly
                pygame.draw.rect(screen, BLACK, pygame.Rect(0,0,WIDTH,SQUARE_SIZE))
                pygame.display.update()
                draw_board(board)
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                        break
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        pos_x = int(event.pos[0] / SQUARE_SIZE)
                        selection = pos_x
            #Once an empty column has been selected, this will pick out the lowest row in that column            
            row = get_next_open_row(board, selection)
            #and this will allow the piece to be dropped at the location
            drop_piece(board, row, selection, piece)
            #Checks if any of the four win-conditions have been acheived
            game_over = win_condition(board, piece)
            #if yes then declares winner and quits the game in 5 seconds
            if game_over:
                print(f"Game Over! Player {piece} wins!")
                draw_board(board)
                label = myfont.render(f"Game Over! Player {piece} wins!", 1, WHITE)
                screen.blit(label, (0, 0))
                pygame.display.update()
                pygame.time.delay(5000)
                pygame.quit()
                sys.exit()
                break
                
